In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
import k1lib, torch.nn as nn

In [3]:
#export
class Lambda(nn.Module):
    def __init__(self, f): super().__init__(); self.f = f
    def forward(self, x): return self.f(x)

In [10]:
#export
class Learner:
    def __init__(self, *args, **kwargs):
        """Initializes a Learner. Intended to have an outside factory function to create this and not be used directly"""
        self.model = None; self.data = None; self.opt = None; self.lossFunction = None
        self.configure(*args, **kwargs)
        self.withCbs(k1lib.Callbacks.standard())
    def configure(self, model:nn.Module, data:k1lib.Data=None, opt=None, lossFunction=None):
        self.model = model or self.model
        self.data = data or self.data
        self.opt = opt or self.opt
        self.lossFunction = lossFunction or self.lossFunction
        return self
    def withCbs(self, cbs): self.cbs = cbs; cbs.injectLearner(self); return self
    def _updateCbsDict(self): self.__dict__.update(self.cbs.dict) # make callback names directly referenceable for quality of life 
    def _run1Batch(self):
        self.cbs("startBatch")
        try:
            self.y = self.model(self.xb); self.cbs("endPass")
            loss = self.lossFunction(self.y, self.yb); self.loss = loss.detach().item(); self.cbs("endLoss")
            if not self.cbs("startBackward"): loss.backward()
            if not self.cbs("startStep"):  self.opt.step()
            if not self.cbs("startZeroGrad"): self.opt.zero_grad()
        except k1lib.CancelBatchException: self.cbs("cancelBatch")
        self.cbs("endBatch")
    def _run1Epoch(self):
        self.cbs("startEpoch")
        try:
            self.nBatches = len(self.data.train); self.model.train()
            for self.batch, (self.xb, self.yb) in enumerate(self.data.train):
                self._run1Batch()
            self.nBatches = len(self.data.valid); self.model.eval(); self.cbs("startValidBatches")
            for self.batch, (self.xb, self.yb) in enumerate(self.data.valid):
                self._run1Batch()
        except k1lib.CancelEpochException: self.cbs("cancelEpoch")
        self.cbs("endEpoch")
    def run(self, epochs, *args, **kwargs):
        """Runs the model for a specified number of epochs."""
        self._updateCbsDict(); self.epochs = epochs; self.configure(None, *args, **kwargs); self.cbs("startRun")
        try:
            for self.epoch in range(epochs):
                self._run1Epoch()
        except k1lib.CancelRunException: self.cbs("cancelRun")
        self.cbs("endRun"); return self
    def _repr_html_(self): return f"""<pre>Model: {klib.textToHtml(str(self.model))}</pre>
    <pre>Optimizer: {klib.textToHtml(str(self.opt))}</pre>{self.cbs._repr_html_()}"""

In [11]:
!exportnb learner.ipynb

/home/kelvin/repos/labs/k1lib/k1lib
Current dir: 0, /home/kelvin/repos/labs/k1lib/export.py
File: /home/kelvin/repos/labs/k1lib/k1lib/learner.py
running bdist_wheel
running build
installing to build/bdist.linux-x86_64/wheel
running install
running install_egg_info
running egg_info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
Copying k1lib.egg-info to build/bdist.linux-x86_64/wheel/k1lib-0.1.0-py3.8.egg-info
running install_scripts
creating build/bdist.linux-x86_64/wheel/k1lib-0.1.0.dist-info/WHEEL
creating 'dist/k1lib-0.1.0-py3-none-any.whl' and adding 'build/bdist.linux-x86_64/wheel' to it
adding 'k1lib-0.1.0.dist-info/METADATA'
adding 'k1lib-0.1.0.dist-info/WHEEL'
adding 'k1lib-0.1.0.dist-info/top_level.txt'
adding 'k1lib-0.1.0.dist-info/RECORD'
removing build/bdist.linux-x86_64

In [33]:
import torch

In [28]:
l = Lambda(lambda x: x[None])

In [32]:
l(torch.rand(2, 3))

tensor([[[0.0042, 0.3004, 0.6085],
         [0.9042, 0.8013, 0.1034]]])

In [10]:
a = {"a": 3, "b": 5}
a.update({"b": 8, "c": 11})

In [11]:
a

{'a': 3, 'b': 8, 'c': 11}